In [ ]:
#stdlib imports
import os
import subprocess
from time import sleep

#third party imports
import numpy as np

In [ ]:
# Test regolith -- Verify the accuracy of intermediate values and model output.
# This script runs REGOLITH with a series preconfigured input files on synthetic topography to compare outputs against
# Analytically computed solutions for several of the models using the same input parameters and topography. 
# After taking the difference between the REGOLITH output and the corresponding analytical depth distribution, 
# the script finds the maximum and minimum values of the residuals.  If the residuals are within a predetermined 
# tolerance, then the output passes the test. Results of the tests are logged below. 

# Get path name of current directory and subdirectories
dir1 = os.path.abspath('.')
dir2 = os.path.abspath('./output')
dir3 = os.path.abspath('./standards')
print(dir2)

# Check for presence of REGOLITH executable file
sys_name = os.name
if sys_name == 'nt':
    cmd = os.path.abspath('./regolith.exe')
    f_stat = os.access(cmd, os.F_OK)
else:
    cmd = os.path.abspath('./regolith')
    f_stat = os.access(cmd, os.F_OK)
print('Executable path name:', cmd)
print('File status, ', f_stat)
if (not f_stat):
    print('Place a copy of the executable file "regolith.exe" \n into ', dir1,' and try again!')

## Iterative process 
# Copy preconfigured input file for each model into 'rg_in.txt' and run REGOLITH for each model
input_files = ['rg_in_LCSD.txt','rg_in_NSD.txt']    #, 'rg_in_NASD.txt','rg_in_NSDA.txt'
for i in input_files:
    subprocess.run(["cp", i, "rg_in.txt"], cwd = dir1)
    try:
        subprocess.run(cmd, cwd = dir1)
    except:
        print('Process failed to run: ', cmd)
    sleep(1)
print('Regolith finished!')
print('--------###---------')
print('*** Test Results ***')

# List output files and analytical standards files 
output_files = ['RG_aspect_gs_Synth.asc', 'RG_del2gs_Synth.asc', 'RG_sec_delta_Synth.asc',\
                'RG_slope.asc', 'RG_LCSD_test_Synth.asc', 'RG_mag_del_z_sq_Synth.asc',\
                'RG_trans_nsd_Synth.asc', 'RG_NSD_test_Synth.asc'] 
standards_files = ['AN_aspect_gs_Synth.asc', 'AN_laplacian_Synth.asc', 'AN_sec_theta_Synth.asc',\
                   'AN_slope_angle_Synth.asc', 'AN_LCSD_Synth.asc', 'AN_mag_grad_sq_Synth.asc',\
                   'AN_nsd_trans_Synth.asc', 'AN_NSD_Synth.asc'] 
names = ['Aspect', 'Laplacian', 'Secant', 'Slope angle', 'LCSD', 'Gradient squared', 'NSD transport', 'NSD']


# Iterate over lists of outputs and standards to compute residuals
# Compare residuals against 'tol'
for o, s, n in zip(output_files, standards_files, names):
    fname1 = os.path.join(dir2, o)
    fname2 = os.path.join(dir3, s)
    fO = np.loadtxt(fname1, dtype='float', skiprows=6) 
    fS = np.loadtxt(fname2, dtype='float', skiprows=6)
    shpfO = np.shape(fO)
    resid = np.zeros(shpfO,dtype=np.float64) # Initialize array for residuals
    resid = fS - fO
    magn = np.mean(np.abs(fS))
    tol = magn*0.004 # Maximum allowable error as fraction of magnitude
    resid_max = np.amax(resid)
    resid_min = np.amin(resid)
    std_dev = np.std(resid)
    mean_resid = np.mean(resid)
#    print('Max, min residuals = ', resid_max, ', ', resid_min)
#    print('Mean of residuals = ', mean_resid)
#    print('tol =', tol)
#    print('Std. deviation of residuals = ', std_dev)
    if (np.abs(mean_resid) > tol):
        print(n, ': Failed') # output_files[0]
    else:
        print(n, ': Passed') #output_files[0]
print('Tests finished!')
